In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
melbdf= pd.read_csv('melb_data.csv')

## Drop columns

- Bedroom2 is from different source and is highly collinear to Rooms. 
- SellerG -more than 200 unique Real Estate Agents.
- Date - lack of summer time data, we cannot assess the correlation between price and the season and use it for price prediction.
- CouncilArea,Postcode,Lattitude,Longtitude - Regionname gives properties location, and Propertycount -Suburb description. 

In [3]:
df=melbdf.drop(columns=['Postcode','Bedroom2','Lattitude','Longtitude','Date','CouncilArea','SellerG'])
df.head()

,Suburb,Address,Rooms,Type,Price,Method,Distance,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,2.5,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,2.5,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,2.5,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,2.5,2.0,1.0,94.0,NaN,NaN,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,2.5,1.0,2.0,120.0,142.0,2014.0,Northern Metropolitan,4019.0


## Filters

S,SA, and SP methods are a complete selling and will be used to filter the dataset. Rows with PI and VB method will be dropped.

In [4]:
df = df.drop(df[(df['Method'] == 'VB') | (df['Method'] == 'PI')].index)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10817 entries, 0 to 13579
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         10817 non-null  object 
 1   Address        10817 non-null  object 
 2   Rooms          10817 non-null  int64  
 3   Type           10817 non-null  object 
 4   Price          10817 non-null  float64
 5   Method         10817 non-null  object 
 6   Distance       10817 non-null  float64
 7   Bathroom       10817 non-null  float64
 8   Car            10774 non-null  float64
 9   Landsize       10817 non-null  float64
 10  BuildingArea   5624 non-null   float64
 11  YearBuilt      6457 non-null   float64
 12  Regionname     10817 non-null  object 
 13  Propertycount  10817 non-null  float64
dtypes: float64(8), int64(1), object(5)
memory usage: 1.2+ MB


The most houses are sold within 2,25M range, so anything beyond this range can be considered as luxury outliers. 

In [5]:
df.drop(df[(df['Price'] >= 2250000)].index, inplace = True)
df.describe()

,Rooms,Price,Distance,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount
count,10314.000000,1.031400e+04,10314.000000,10314.000000,10273.000000,10314.000000,5353.000000,6146.000000,10314.000000
mean,2.871243,9.669102e+05,10.558619,1.452589,1.572179,514.553907,144.976173,1965.708266,7429.377545
std,0.885088,4.227061e+05,6.151629,0.614738,0.938050,1441.252115,619.897555,34.965783,4485.622685
min,1.000000,1.850000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,1830.000000,249.000000
25%,2.000000,6.450000e+05,6.400000,1.000000,1.000000,177.250000,92.000000,1950.000000,4181.000000
50%,3.000000,8.795000e+05,9.900000,1.000000,1.000000,430.000000,121.000000,1970.000000,6543.000000
75%,3.000000,1.240000e+06,13.500000,2.000000,2.000000,640.000000,160.000000,1998.000000,10175.000000
max,8.000000,2.248000e+06,48.100000,6.000000,10.000000,76000.000000,44515.000000,2018.000000,21650.000000


Cleaning duplicated rows in Address column. As shown in the Address section of the data exploration stage the same properties seem to have different selling dates, price or features. As it cannot be reasonably explained hence duplicates will be dropped.

In [6]:
# Check and drop duplicated rows
df.Address.describe(exclude=None)

count            10314
unique           10229
top       5 Charles St
freq                 3
Name: Address, dtype: object

In [7]:
df = df.drop_duplicates(subset=["Address"],keep=False)
df.Address.describe(exclude=None)

count          10147
unique         10147
top       9 Rowan St
freq               1
Name: Address, dtype: object

In [8]:
# drop Address and Suburb as we will not use them further
df=df.drop(columns=['Address','Suburb'])
df.head()

,Rooms,Type,Price,Method,Distance,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname,Propertycount
0,2,h,1480000.0,S,2.5,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan,4019.0
1,2,h,1035000.0,S,2.5,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan,4019.0
5,2,h,941000.0,S,2.5,1.0,0.0,181.0,NaN,NaN,Northern Metropolitan,4019.0
6,3,h,1876000.0,S,2.5,2.0,0.0,245.0,210.0,1910.0,Northern Metropolitan,4019.0
7,2,h,1636000.0,S,2.5,1.0,2.0,256.0,107.0,1890.0,Northern Metropolitan,4019.0


## Missing and Null values

In [9]:
# check for missing values
df.isnull().sum()

Rooms               0
Type                0
Price               0
Method              0
Distance            0
Bathroom            0
Car                39
Landsize            0
BuildingArea     4894
YearBuilt        4110
Regionname          0
Propertycount       0
dtype: int64

In [10]:
# check for null values
print('Number of zeros in "Car":', (df['Car'] == 0.0).sum())
print('Number of zeros in "Bathroom":', (df['Bathroom'] == 0.0).sum())
print('Number of zeros in "BuildingArea":', (df['BuildingArea'] == 0.0).sum())

Number of zeros in "Car": 791
Number of zeros in "Bathroom": 25
Number of zeros in "BuildingArea": 12


In [11]:
# creat new column with building age
# replace all missing values with median building age
df['Build_Age']=2022-df['YearBuilt']
df['Build_Age'] = df['Build_Age'].fillna(df['Build_Age'].median())
df['Build_Age'].describe()

count    10147.000000
mean        54.391052
std         27.020672
min          4.000000
25%         47.000000
50%         52.000000
75%         62.000000
max        192.000000
Name: Build_Age, dtype: float64

Building age range looks correct

In [12]:
# YearBuilt can be dropped
df=df.drop(columns=['YearBuilt'])

In [13]:
# replace missing values in building area with mean by type
type_h=df[df['Type']=='h']
type_u=df[df['Type']=='u']
type_t=df[df['Type']=='t']
df.loc[df['Type']=='h','BuildingArea'] = df.loc[df['Type']=='h', 'BuildingArea'].fillna(type_h['BuildingArea'].mean())
df.loc[df['Type']=='u','BuildingArea'] = df.loc[df['Type']=='u', 'BuildingArea'].fillna(type_u['BuildingArea'].mean())
df.loc[df['Type']=='t','BuildingArea'] = df.loc[df['Type']=='t', 'BuildingArea'].fillna(type_t['BuildingArea'].mean())

In [14]:
#check building area nulls
df.loc[df['BuildingArea']==0.0] 

,Rooms,Type,Price,Method,Distance,Bathroom,Car,Landsize,BuildingArea,Regionname,Propertycount,Build_Age
4344,2,t,841000.0,S,2.3,1.0,1.0,215.0,0.0,Northern Metropolitan,6821.0,22.0
12226,5,h,1950000.0,S,9.7,3.0,2.0,743.0,0.0,Southern Metropolitan,7809.0,73.0
12249,3,h,814000.0,S,12.1,1.0,2.0,542.0,0.0,Northern Metropolitan,10175.0,52.0
12395,4,h,570000.0,S,20.6,2.0,2.0,504.0,0.0,Northern Metropolitan,5833.0,22.0
13040,3,h,1390000.0,S,4.6,2.0,1.0,125.0,0.0,Southern Metropolitan,7717.0,20.0
13207,3,h,1205000.0,SA,12.3,2.0,4.0,622.0,0.0,Southern Metropolitan,768.0,62.0
13370,3,h,1650000.0,SP,10.3,2.0,2.0,623.0,0.0,Southern Metropolitan,6938.0,102.0
13380,4,h,815000.0,S,12.1,2.0,1.0,525.0,0.0,Northern Metropolitan,10175.0,57.0
13402,3,h,412500.0,S,20.6,2.0,1.0,197.0,0.0,Northern Metropolitan,15510.0,10.0
13411,3,h,595000.0,S,19.6,1.0,2.0,536.0,0.0,Northern Metropolitan,10926.0,42.0


In [15]:
#all null values are with house&t type, will be replaced with type h mean
condition = (df['BuildingArea']== 0)
df.loc[condition, 'BuildingArea'] = type_h['BuildingArea'].mean()

In [16]:
#replace missing values in car with median
df['Car'] = df['Car'].fillna(df['Car'].median())

In [17]:
#replace null car with median for real estate with a distance from the center of more than 3 km 
condition  = ((df['Type']=='h') & (df['Car']==0.0) & (df['Distance'] > 3))
df.loc[condition, 'Car'] = df['Car'].median()

In [18]:
# replace null values in bathroom with median
condition = (df['Bathroom']== 0)
df.loc[condition,'Bathroom'] = df['Bathroom'].median()

In [19]:
# check for Nan values again
df.isnull().sum()

Rooms            0
Type             0
Price            0
Method           0
Distance         0
Bathroom         0
Car              0
Landsize         0
BuildingArea     0
Regionname       0
Propertycount    0
Build_Age        0
dtype: int64

In [20]:
# check for null values again
print('Number of zeros in "Car":', (df['Car'] == 0.0).sum())
print('Number of zeros in "Bathroom":', (df['Bathroom'] == 0.0).sum())
print('Number of zeros in "BuildingArea":', (df['BuildingArea'] == 0.0).sum())

Number of zeros in "Car": 249
Number of zeros in "Bathroom": 0
Number of zeros in "BuildingArea": 0


Convert categorical variables into dummy/indicator variables.

In [21]:
df_final=pd.get_dummies(df).reset_index(drop=True)
df_final

,Rooms,Price,Distance,Bathroom,Car,Landsize,BuildingArea,Propertycount,Build_Age,Type_h,...,Method_SA,Method_SP,Regionname_Eastern Metropolitan,Regionname_Eastern Victoria,Regionname_Northern Metropolitan,Regionname_Northern Victoria,Regionname_South-Eastern Metropolitan,Regionname_Southern Metropolitan,Regionname_Western Metropolitan,Regionname_Western Victoria
0,2,1480000.0,2.5,1.0,1.0,202.0,167.281095,4019.0,52.0,1,...,0,0,0,0,1,0,0,0,0,0
1,2,1035000.0,2.5,1.0,0.0,156.0,79.000000,4019.0,122.0,1,...,0,0,0,0,1,0,0,0,0,0
2,2,941000.0,2.5,1.0,0.0,181.0,167.281095,4019.0,52.0,1,...,0,0,0,0,1,0,0,0,0,0
3,3,1876000.0,2.5,2.0,0.0,245.0,210.000000,4019.0,112.0,1,...,0,0,0,0,1,0,0,0,0,0
4,2,1636000.0,2.5,1.0,2.0,256.0,107.000000,4019.0,132.0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10142,3,582000.0,16.5,2.0,2.0,256.0,167.281095,2474.0,52.0,1,...,0,0,0,0,1,0,0,0,0,0
10143,4,1245000.0,16.7,2.0,2.0,652.0,167.281095,7392.0,41.0,1,...,0,0,0,0,0,0,1,0,0,0
10144,3,1031000.0,6.8,2.0,2.0,333.0,133.000000,6380.0,27.0,1,...,0,1,0,0,0,0,0,0,1,0
10145,3,1170000.0,6.8,2.0,4.0,436.0,167.281095,6380.0,25.0,1,...,0,0,0,0,0,0,0,0,1,0


In [22]:
#export the finished dataframe so that it can be used by other notebooks
df_final.to_csv('final_df.csv', index=False)

In [23]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10147 entries, 0 to 10146
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Rooms                                  10147 non-null  int64  
 1   Price                                  10147 non-null  float64
 2   Distance                               10147 non-null  float64
 3   Bathroom                               10147 non-null  float64
 4   Car                                    10147 non-null  float64
 5   Landsize                               10147 non-null  float64
 6   BuildingArea                           10147 non-null  float64
 7   Propertycount                          10147 non-null  float64
 8   Build_Age                              10147 non-null  float64
 9   Type_h                                 10147 non-null  uint8  
 10  Type_t                                 10147 non-null  uint8  
 11  Ty

In [24]:
#export the original dataframe so that it can be used by other notebooks
df.to_csv('origin_df.csv', index=False)